In [1]:
import spacy
from spacy.training.example import Example
from pathlib import Path
import random 
import plac



In [2]:

'''nlp = spacy.load("en_core_web_sm")
sent_0 = nlp(u'Myriam saw Clement with a telescope.')
sent_1 = nlp(u'Self-driving cars shift insurance liability toward manufacturers.')
sent_2 = nlp(u'I shot the elephant in my pyjamas.')
for chunk in sent_0.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
for token in sent_0:
    print("--",token.text, token.dep_, token.head.text, token.head.pos_,
    [child for child in token.children])'''



'nlp = spacy.load("en_core_web_sm")\nsent_0 = nlp(u\'Myriam saw Clement with a telescope.\')\nsent_1 = nlp(u\'Self-driving cars shift insurance liability toward manufacturers.\')\nsent_2 = nlp(u\'I shot the elephant in my pyjamas.\')\nfor chunk in sent_0.noun_chunks:\n    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)\nfor token in sent_0:\n    print("--",token.text, token.dep_, token.head.text, token.head.pos_,\n    [child for child in token.children])'

In [11]:
nlp = spacy.load("en_core_web_sm")
# Load a blank English model
nlp = spacy.blank("en")


# Add a dependency parser to the pipeline
# add the parser to the pipeline if it doesn't exist
if 'parser' not in nlp.pipe_names:
    parser = nlp.create_pipe('parser')
    nlp.add_pipe('parser', first=True)
 # otherwise, get it, so we can add labels to it
else:
    parser = nlp.get_pipe('parser')

tags=["amod"]
# Define training data with examples of the new "Quantity" dependency
TRAINING_DATA = [
    ("Few people attended the meeting.", {'heads': [1, 1, 1, 3,3,1], 'deps': ['amod', 'nsubj', 'ROOT', 'det', 'osubj','punct']}),
    ("Many students passed the exam.", {'heads': [1, 1, 1, 3,3,1], 'deps': ['amod', 'nsubj', 'ROOT', 'det', 'osubj','punct']}),
    #("There were enough chairs for everyone.", {"heads": [2, 2, 4, 4, 5, 5], "deps": ["expl", "ROOT", "amod", "nsubj", "prep", "pobj"]}),
    ("Some students completed all the assignments.", {'heads': [1, 1, 1, 3, 3, 3,1], 'deps': ['amod', 'nsubj', 'ROOT', 'det', 'det', 'osubj', 'punct']}),
    #("Half of the pizza is gone.", {"heads": [1, 1, 1, 5], "deps": ["amod", "prep", "pobj", "ROOT"]}),
    #("The whole team worked together.", {"heads": [1, 3, 3, 3], "deps": ["det", "amod", "ROOT", "advmod"]}),
    #("Numerous books filled the shelves.", {"heads": [1, 1, 3, 3, 3], "deps": ["amod", "nsubj", "ROOT", "det", "dobj"]}),
    #("They trade mortgage-backed securities.", {'heads': [1, 1, 4, 4, 5, 1, 1],'deps': ['nsubj', 'ROOT', 'compound', 'punct', 'nmod', 'dobj','punct']}),

    # Add more examples as needed
]

# Train the dependency parser
for _, annotations in TRAINING_DATA:
    for dep in annotations.get('deps', []):
        parser.add_label(dep)
# get names of other pipes to disable them during training
other_pipess = [pipe for pipe in nlp.pipe_names if pipe != 'parser']
with nlp.disable_pipes(*other_pipess): # only train parser
    optimizer1 = nlp.create_optimizer()
    for itn in range(20):
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in TRAINING_DATA:
            doc = nlp.make_doc(text)
            exampl = Example.from_dict(doc, annotations)
            nlp.update([exampl], sgd=optimizer1,losses=losses)
            #nlp.update([text], [annotations], sgd=optimizer1)
        #print(losses)


# test the trained model
test_text = "Few people attended the meeting."
doc = nlp(test_text)
print('Dependencies', [(t.text, t.dep_, t.head.text) for t in doc])
# save model to output directory
output_dir = Path("./models")
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
    # test the saved model
    print("Loading from", output_dir)
    nlp22 = spacy.load(output_dir)
    doc = nlp22(test_text)
    print('Dependencies', [(t.text, t.dep_, t.head.text) for t in doc])






KeyError: 2